In [1]:
# импортируем функцию обучения и функцию предсказания
from main import matching_training, matching_predict

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import torch
import transformers
from scipy.spatial.distance import cdist
import pandas as pd
import re
import numpy as np
from tqdm import notebook
import pickle


In [3]:
mp = pd.read_csv('/content/marketing_product.csv', delimiter=';')
mdp = pd.read_csv('/content/marketing_dealerprice.csv', delimiter=';')
mpdk = pd.read_csv('/content/marketing_productdealerkey.csv', delimiter=';')

# деление данных на обучение и тест
train_mdp, tst_mdp = train_test_split(mdp, test_size=0.3)

# перевод фрэймов в список словарей
ld_mp = mp.to_dict('records')
ld_mdp_train = train_mdp.to_dict('records')
ld_mdp_tst = tst_mdp.to_dict('records')
ld_mpdk = mpdk.to_dict('records')

# обучение модели (столбец из данных производителя,
# по которому ищем сходство со столбцом
# описаний товаров product_name из таблицы диллеров, по умолчанию
# равен name, но можно подставить к примеру nm='name_1c')
%time model_embeddings = matching_training(ld_mp, ld_mdp_train, ld_mpdk)

# сохраняем модель и фрэйм с эмбеддингами данных производителя в файлы
Pkl_Filename = "Pickle_Model.pkl"
with open(Pkl_Filename, 'wb') as file:
    pickle.dump(model_embeddings[0], file)
model_embeddings[1].to_csv('embeddings.csv')

# получение предсказания (k по умолчанию равно 5, nm по умолчанию равно name,
# оставляем nm=name, так как обучение модели было именно по столбцу name)
%time res = matching_predict(ld_mp, ld_mdp_tst, model_embeddings)

df_res = pd.DataFrame(res)
df_res.head()

/content/main.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name + '_tok'] = name_tok
/content/main.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_mp_name.drop([nm, 'id'], axis=1, inplace=True)


tokenizer_config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

...

  0%|          | 0/399 [00:00<?, ?it/s]

CPU times: user 32min 31s, sys: 14.5 s, total: 32min 46s
Wall time: 30min 41s


  0%|          | 0/191 [00:00<?, ?it/s]

CPU times: user 12min 30s, sys: 4.86 s, total: 12min 35s
Wall time: 12min 36s


,1,2,3,4,5
0,169,170,168,190,80
1,456,181,440,222,188
2,263,264,217,501,287
3,19,453,77,78,20
4,251,273,252,277,250


In [4]:
# Получение метрики recall@k (или accuracy)
# на вход функции подаётся тестовый фрэйм, фрэйм данных с внешними ключами,
# фрэйм результата функции предсказания, колличество k, по которому
# считается метрика recall@k
# возвращается таблица со столбцами id карточек, верного id и recall@k
def recall_k(tst, mpdk, df_res, k):
    test = tst.merge(mpdk,
                     how='left',
                     left_on=['product_key', 'dealer_id'],
                     right_on=['key', 'dealer_id']).loc[:, ['product_key',
                                             'key',
                                             'product_id']]
    test = pd.concat([test, df_res], axis=1)
    test = test.loc[~test.product_id.isnull()]
    test.reset_index(drop=True, inplace=True)

    for i in range(test.shape[0]):
        if test.loc[i, 'product_id'] in test.loc[
            i, [str(t) for t in range(1, k + 1)]].values:
            test.loc[i, f'recall@{str(k)}'] = 1
        else:
            test.loc[i, f'recall@{str(k)}'] = 0
    return test

k = df_res.shape[1]
test = recall_k(tst_mdp, mpdk, df_res, k)
print('Accuracy : ', sum(test[f'recall@{str(k)}']) / test.shape[0])

Accuracy :  0.9848398711389047


In [5]:
# ещё пример с предсказанием
# так как обучали на train_mdp, берём 30 последних строчек из tst_mdp
tst_mdp_ex = tst_mdp.tail(30)
ld_mdp_tst_ex = tst_mdp_ex.to_dict('records')

# загружаем сохранённую модель и фрэйм с эмбеддингами данных производителя
with open(Pkl_Filename, 'rb') as file:
    model_ex = pickle.load(file)
embeddings_ex = pd.read_csv('embeddings.csv', index_col=[0])

# функция предсказания (так как модель обучалась на данных столбца name,
# не проставляем параметр nm, который по умолчаию равен name,
# колличество карточек k поставим раным 3,
# третьим параметром передаём модель и эмбеддинги данных производителя)
%time res_ex = matching_predict(ld_mp, ld_mdp_tst_ex, (model_ex, embeddings_ex), k=3)
df_res_ex = pd.DataFrame(res_ex)



0it [00:00, ?it/s]

CPU times: user 8.14 s, sys: 4.91 s, total: 13 s
Wall time: 15.2 s


In [6]:
# Получение метрики recall@k (или accuracy)
k = df_res_ex.shape[1]
test_ex = recall_k(tst_mdp_ex, mpdk, df_res_ex, k)
print('Accuracy : ', sum(test_ex[f'recall@{str(k)}']) / test_ex.shape[0])

Accuracy :  0.9615384615384616
